In [1]:
import matplotlib.pyplot as plt
import librosa
import numpy as np
import pandas as pd
import os
import IPython.display as ipd
from IPython.display import clear_output
%matplotlib inline

In [2]:
root_path="C:/Users/sarib/Desktop/Gohm/SpeechRecognition/database"

In [3]:
def mfcc_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    
    return mfccs_scaled_features

In [4]:
#NOT USED
def fft_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    fft = np.fft.fft(signal)
    
    return fft

In [5]:
#NOT USED
signal, sample_rate = librosa.load('database/ac/ac_AHCG_FSRPNCW.wav')
ipd.Audio('database/ac/ac_AHCG_FSRPNCW.wav')
fft = np.fft.fft(signal)
fft

array([ 3.96959355+0.j        , -1.57368256-0.43014124j,
       -0.5562213 +0.51962332j, ...,  0.00995858+0.12657352j,
       -0.5562213 -0.51962332j, -1.57368256+0.43014124j])

In [6]:
#NOT USED
signal, sample_rate = librosa.load('database\sol\sol_AKZI_FAWCVUT.wav' )
ipd.Audio('database\sol\sol_AKZI_FAWCVUT.wav' )
mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
mfccs_features.shape

(40, 44)

In [7]:
"""from tqdm import tqdm
import time"""
extracted_feataures =[]
for root, directories, files in os.walk(root_path):
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names))
            data = mfcc_extractor(os.path.join(root,file_names))
            extracted_feataures.append([data,os.path.basename(os.path.normpath(root))])

    """for i in tqdm (range (100), desc="Loading..."):
        time.sleep(3)"""

In [8]:
extracted_feataures_dataframe= pd.DataFrame(extracted_feataures, columns=["mfcc", "word"])
extracted_feataures_dataframe.head()

,mfcc,word
0,"[-483.0338, 62.193104, -10.424236, 17.89255, -...",ac
1,"[-393.0876, 61.467396, -20.944004, 18.326431, ...",ac
2,"[-557.86304, 99.75818, -6.094904, 0.65864086, ...",ac
3,"[-518.9073, 52.873825, -3.625966, 5.807297, -9...",ac
4,"[-539.5348, 84.21058, -6.7271714, 4.8634925, -...",ac


In [9]:
X=np.array(extracted_feataures_dataframe['mfcc'].tolist())
Y=np.array(extracted_feataures_dataframe['word'].tolist())

In [10]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
Y_numerical=to_categorical(labelencoder.fit_transform(Y))

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y_numerical, test_size=0.2, random_state=0)

In [12]:
#NOT USED
X_train.shape

(21153, 40)

In [13]:
#NOT USED
len(Y_test)

5289

ANN Model

In [14]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,CSVLogger
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [15]:
model= Sequential()

model.add(Flatten(input_shape=(40, )))
model.add(Dense(300, Activation('relu')))
model.add(Dropout(0.2))
model.add(Dense(300, Activation('relu')))
model.add(Dense(Y_numerical.shape[1], Activation('softmax')))


"""
model.add(Dense(200, input_shape=(40,)))
model.add(Activation('relu'))

model.add(Dense(300))
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Activation('relu'))

model.add(Dense(Y_numerical.shape[1]))
model.add(Activation('sigmoid'))"""

"\nmodel.add(Dense(200, input_shape=(40,)))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(300))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(50))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(Y_numerical.shape[1]))\nmodel.add(Activation('sigmoid'))"

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 40)                0         
                                                                 
 dense (Dense)               (None, 300)               12300     
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 300)               90300     
                                                                 
 dense_2 (Dense)             (None, 14)                4214      
                                                                 
Total params: 106,814
Trainable params: 106,814
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam())

In [18]:
"""checkpointpath="training3/cp.ckpt"
checkpointer = ModelCheckpoint(checkpointpath, monitor='val_accuracy',
                               verbose=2, save_best_only=True, mode='max')

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=2)
callbackList= [checkpointer, early_stop]
model.fit(X_train, Y_train, epochs=200, validation_data=(X_test, Y_test), callbacks=callbackList, verbose=2)"""



'checkpointpath="training3/cp.ckpt"\ncheckpointer = ModelCheckpoint(checkpointpath, monitor=\'val_accuracy\',\n                               verbose=2, save_best_only=True, mode=\'max\')\n\nearly_stop = EarlyStopping(monitor=\'val_loss\', patience=10, verbose=2)\ncallbackList= [checkpointer, early_stop]\nmodel.fit(X_train, Y_train, epochs=200, validation_data=(X_test, Y_test), callbacks=callbackList, verbose=2)'

In [19]:
"""checkpointpath="C:/Users/sarib/Desktop/Gohm/SpeechRecognition/saved_models/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpointer = ModelCheckpoint(checkpointpath, monitor='val_accuracy',
                               verbose=2, save_best_only=True, mode='max')

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=2)
log_csv = CSVLogger('csv_logs.csv', separator=',', append=False)
callbackList= [checkpointer, early_stop, log_csv]
pretrained_model=model.fit(X_train, Y_train, epochs=200, validation_data=(X_test, Y_test), callbacks=callbackList, verbose=2)
model.save('savedModel.h5', overwrite = True, include_optimizer = True)"""


checkpointpath="C:/Users/sarib/Desktop/Gohm/SpeechRecognition/saved_models/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpointer = ModelCheckpoint(checkpointpath, monitor='val_accuracy',
                               verbose=2, save_best_only=True, mode='max')

#early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=2)
callbackList= [checkpointer]
pretrained_model=model.fit(X_train, Y_train, epochs=200, validation_data=(X_test, Y_test), callbacks=callbackList, verbose=2)
model.save('completesavedModel/')

Epoch 1/200

Epoch 1: val_accuracy improved from -inf to 0.47533, saving model to C:/Users/sarib/Desktop/Gohm/SpeechRecognition/saved_models\weights-improvement-01-0.48.hdf5
662/662 - 3s - loss: 4.3060 - accuracy: 0.2706 - val_loss: 1.5406 - val_accuracy: 0.4753 - 3s/epoch - 4ms/step
Epoch 2/200

Epoch 2: val_accuracy improved from 0.47533 to 0.50747, saving model to C:/Users/sarib/Desktop/Gohm/SpeechRecognition/saved_models\weights-improvement-02-0.51.hdf5
662/662 - 2s - loss: 1.6320 - accuracy: 0.4405 - val_loss: 1.4259 - val_accuracy: 0.5075 - 2s/epoch - 3ms/step
Epoch 3/200

Epoch 3: val_accuracy improved from 0.50747 to 0.53110, saving model to C:/Users/sarib/Desktop/Gohm/SpeechRecognition/saved_models\weights-improvement-03-0.53.hdf5
662/662 - 2s - loss: 1.4800 - accuracy: 0.4874 - val_loss: 1.3410 - val_accuracy: 0.5311 - 2s/epoch - 3ms/step
Epoch 4/200

Epoch 4: val_accuracy improved from 0.53110 to 0.57799, saving model to C:/Users/sarib/Desktop/Gohm/SpeechRecognition/saved_mo

INFO:tensorflow:Assets written to: completesavedModel/assets


INFO:tensorflow:Assets written to: completesavedModel/assets


In [20]:
#model.save('pretrained_model.h5')

In [21]:
print("Test Score: ", model.evaluate(X_test,Y_test))
print("Train Score: ", model.evaluate(X_train,Y_train))

166/166 [==============================] - 1s 3ms/step - loss: 1.0255 - accuracy: 0.7523
Test Score:  [1.0254874229431152, 0.7523161172866821]
662/662 [==============================] - 2s 3ms/step - loss: 0.1273 - accuracy: 0.9597
Train Score:  [0.12732966244220734, 0.9597220420837402]


In [32]:
from keras.models import load_model, save_model
pretrained_model = load_model('completesavedModel/')
#model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam())

In [23]:
"""plt.plot(model.history['loss'], label='train')
plt.plot(model.history['val_loss'], label='test')
plt.legend()
plt.show()"""

"plt.plot(model.history['loss'], label='train')\nplt.plot(model.history['val_loss'], label='test')\nplt.legend()\nplt.show()"

In [39]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=2)
callbackListPre= [checkpointer, early_stop]
pretrained_model.fit(X_train, Y_train, epochs=200, validation_data=(X_test, Y_test), callbacks=callbackListPre, verbose=2)

Epoch 1/200

Epoch 1: val_accuracy did not improve from 0.77179
662/662 - 2s - loss: 0.3722 - accuracy: 0.8789 - val_loss: 0.9798 - val_accuracy: 0.7684 - 2s/epoch - 3ms/step
Epoch 2/200

Epoch 2: val_accuracy did not improve from 0.77179
662/662 - 2s - loss: 0.3826 - accuracy: 0.8758 - val_loss: 0.9897 - val_accuracy: 0.7542 - 2s/epoch - 3ms/step
Epoch 3/200

Epoch 3: val_accuracy did not improve from 0.77179
662/662 - 2s - loss: 0.3608 - accuracy: 0.8829 - val_loss: 0.9724 - val_accuracy: 0.7529 - 2s/epoch - 3ms/step
Epoch 4/200

Epoch 4: val_accuracy did not improve from 0.77179
662/662 - 2s - loss: 0.3749 - accuracy: 0.8770 - val_loss: 0.9700 - val_accuracy: 0.7584 - 2s/epoch - 3ms/step
Epoch 5/200

Epoch 5: val_accuracy did not improve from 0.77179
662/662 - 2s - loss: 0.3649 - accuracy: 0.8822 - val_loss: 1.0210 - val_accuracy: 0.7557 - 2s/epoch - 3ms/step
Epoch 6/200

Epoch 6: val_accuracy did not improve from 0.77179
662/662 - 2s - loss: 0.3761 - accuracy: 0.8778 - val_loss: 0.

In [41]:
pretrained_model2 = load_model('completesavedModel/')

In [42]:
print("Test Score: ", pretrained_model2.evaluate(X_test,Y_test))
print("Train Score: ", pretrained_model2.evaluate(X_train,Y_train))

166/166 [==============================] - 1s 2ms/step - loss: 0.9957 - accuracy: 0.7523
Test Score:  [0.9957082867622375, 0.7523161172866821]
662/662 [==============================] - 2s 2ms/step - loss: 0.1273 - accuracy: 0.9597
Train Score:  [0.12732966244220734, 0.9597220420837402]


In [26]:
#model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam())

PREDICTION WITH DATA

In [43]:
predictiondata_feataures =[]
counter_true=0
for root, directories, files in os.walk("C:/Users/sarib/Desktop/Gohm/SpeechRecognition/RecordsFromDataset"): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            r = mfcc_extractor(os.path.join(root,file_names))
            rl=r.reshape(1,-1)
            predicted_label=np.argmax(pretrained_model2.predict(rl), axis=-1)
            prediction_class = labelencoder.inverse_transform(predicted_label) 
            predictiondata_feataures.append([os.path.basename(os.path.normpath(root)), prediction_class, predicted_label])
            if os.path.basename(os.path.normpath(root))==prediction_class:
                counter_true+=1

In [44]:
predictiondata_feataures_dataframe= pd.DataFrame(predictiondata_feataures, columns=["actual data", "predicted data", "predicted data code"])
percentage= (counter_true/len(predictiondata_feataures))*100
print("The percantage of the true predicted data: ", percentage,"%")
#predictiondata_feataures_dataframe.to_csv("output.csv")
predictiondata_feataures_dataframe


The percantage of the true predicted data:  61.904761904761905 %


,actual data,predicted data,predicted data code
0,ac,[ac],[0]
1,ac,[dur],[4]
2,ac,[iptal],[9]
3,asagi,[evet],[5]
4,asagi,[evet],[5]
5,asagi,[baslat],[2]
6,baslat,[baslat],[2]
7,baslat,[baslat],[2]
8,baslat,[baslat],[2]
9,devam,[devam],[3]


In [45]:
predictiondatas_feataures =[]
counters_true=0
for root, directories, files in os.walk("C:/Users/sarib/Desktop/Gohm/SpeechRecognition/Records"): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            r = mfcc_extractor(os.path.join(root,file_names))
            rl=r.reshape(1,-1)
            predicteds_label=np.argmax(pretrained_model2.predict(rl), axis=-1)
            predictions_class = labelencoder.inverse_transform(predicteds_label) 
            predictiondatas_feataures.append([os.path.basename(os.path.normpath(root)), predictions_class, predicteds_label])
            if os.path.basename(os.path.normpath(root))==predictions_class:
                counters_true+=1

In [46]:
predictiondatas_feataures_dataframe= pd.DataFrame(predictiondatas_feataures, columns=["actual data", "predicted data", "predicted data code"])
percentages= (counters_true/len(predictiondatas_feataures))*100
print("The percantage of the true predicted data: ", percentages,"%")
#predictiondata_feataures_dataframe.to_csv("output.csv")
predictiondatas_feataures_dataframe


The percantage of the true predicted data:  47.61904761904761 %


,actual data,predicted data,predicted data code
0,ac,[ac],[0]
1,ac,[ac],[0]
2,ac,[asagi],[1]
3,asagi,[asagi],[1]
4,asagi,[asagi],[1]
5,asagi,[asagi],[1]
6,baslat,[baslat],[2]
7,baslat,[baslat],[2]
8,baslat,[baslat],[2]
9,devam,[devam],[3]


In [31]:
"""filename="predictionData/dursultan.wav"
r=mfcc_extractor(filename)
print(r)
rl=r.reshape(1,-1)
print(rl)
print(rl.shape)
predicted_label=np.argmax(model.predict(rl), axis=-1)

print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class"""

'filename="predictionData/dursultan.wav"\nr=mfcc_extractor(filename)\nprint(r)\nrl=r.reshape(1,-1)\nprint(rl)\nprint(rl.shape)\npredicted_label=np.argmax(model.predict(rl), axis=-1)\n\nprint(predicted_label)\nprediction_class = labelencoder.inverse_transform(predicted_label) \nprediction_class'